# 第4回課題
内包表記，並列処理

In [1]:
#!pip install joblib

In [2]:
from joblib import Parallel, delayed
import numpy as np
import pprint
pp = pprint.PrettyPrinter(indent=4)

## 内包表記
python における高速化テクの1つである内包表記の実装（for 文の削減）

### 課題3

以下の for 文を内包表記に書き直しなさい．
```[python]
data = []
for i in range(5):
    for j in range(4):
        data.append(i*j)
data
```
出力：
[0, 0, 0, 0, 0, 1, 2, 3, 0, 2, 4, 6, 0, 3, 6, 9, 0, 4, 8, 12]

In [5]:
[i*j for i in range(5) for j in range(4)]

[0, 0, 0, 0, 0, 1, 2, 3, 0, 2, 4, 6, 0, 3, 6, 9, 0, 4, 8, 12]

### 課題2

以下の for 文を内包表記に書き直しなさい．
```[python]
data = []
for i in range(5):
    inner = []
    for j in range(i, 6):
        inner.append(i)
    data.append(inner)
data
```
出力：
[[0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1], [2, 2, 2, 2], [3, 3, 3], [4, 4]]

In [6]:
[[i for j in range(i, 6)] for i in range(5)]

[[0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1], [2, 2, 2, 2], [3, 3, 3], [4, 4]]

## 並列処理
python における高速化テクの1つである並列化の実装（複数コア，CPU の使用）

joblib だけでなく multiprocessing もある

### 例：2乗の場合

In [7]:
%%timeit 3 # 3回下記のコードを実行した時の時間を表示
r = []
for i in range(10000):
    r.append(i ** 2)
r[:3] + r[-3:]

3 ms ± 146 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
?Parallel

In [9]:
%%timeit 3 
power = lambda x: x**2
r = Parallel(n_jobs=-1, verbose=0)([delayed(power)(i) for i in range(10000)])
r[:3] + r[-3:]

352 ms ± 19.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
power = lambda x: x**2
r = Parallel(n_jobs=-1, verbose=3)( [delayed(power)(i) for i in range(10000)] )
r[:3] + r[-3:]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 2032 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 10000 out of 10000 | elapsed:    0.3s finished


[0, 1, 4, 99940009, 99960004, 99980001]

In [11]:
%%timeit 3
# numpy が最速というのはよくある
np.arange(10000) ** 2

11.4 µs ± 276 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [12]:
%%timeit 3
np.fromiter(range(10000), dtype=np.int) ** 2

247 µs ± 12 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### 課題1

x = [(i, i+1) for i in range(100000)] としたとき，各 index 毎に積を求めよ

出力 [0, 2, 6,（中略） 9999500006, 9999700002, 9999900000]

In [16]:
x = lambda i, j: i * j
Parallel(n_jobs = -1)([delayed(x)(i, i+1) for i in range(100000)])

[0,
 2,
 6,
 12,
 20,
 30,
 42,
 56,
 72,
 90,
 110,
 132,
 156,
 182,
 210,
 240,
 272,
 306,
 342,
 380,
 420,
 462,
 506,
 552,
 600,
 650,
 702,
 756,
 812,
 870,
 930,
 992,
 1056,
 1122,
 1190,
 1260,
 1332,
 1406,
 1482,
 1560,
 1640,
 1722,
 1806,
 1892,
 1980,
 2070,
 2162,
 2256,
 2352,
 2450,
 2550,
 2652,
 2756,
 2862,
 2970,
 3080,
 3192,
 3306,
 3422,
 3540,
 3660,
 3782,
 3906,
 4032,
 4160,
 4290,
 4422,
 4556,
 4692,
 4830,
 4970,
 5112,
 5256,
 5402,
 5550,
 5700,
 5852,
 6006,
 6162,
 6320,
 6480,
 6642,
 6806,
 6972,
 7140,
 7310,
 7482,
 7656,
 7832,
 8010,
 8190,
 8372,
 8556,
 8742,
 8930,
 9120,
 9312,
 9506,
 9702,
 9900,
 10100,
 10302,
 10506,
 10712,
 10920,
 11130,
 11342,
 11556,
 11772,
 11990,
 12210,
 12432,
 12656,
 12882,
 13110,
 13340,
 13572,
 13806,
 14042,
 14280,
 14520,
 14762,
 15006,
 15252,
 15500,
 15750,
 16002,
 16256,
 16512,
 16770,
 17030,
 17292,
 17556,
 17822,
 18090,
 18360,
 18632,
 18906,
 19182,
 19460,
 19740,
 20022,
 20306,
 

### 発展課題
C = np.cumsum(np.arange(1000)) の各項の差を求めよ

出力[1, 2, 3,（中略） 997, 998, 999] = np.arange(1, 1000)

In [22]:
C = np.cumsum(np.arange(1000)) 
x = lambda i, j: j - i
Parallel(n_jobs = -1)([delayed(x)(C[i], C[i+1]) for i in range(999)])

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185